In [1]:
from math import pi as PI
import torch
from torch import nn, optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import os
import ast
import matplotlib.ticker as ticker

from PINN.diff import diff
from PINN.networks import FCNN,Resnet
from PINN.generators import generator_2dspatial_rectangle, generator_2dspatial_segment
from PINN.conditions import BoundaryCondition
from PINN.solvers import SingleNetworkApproximator2DSpatial_heat,_solve_2dspatial
from PINN.monitors import Monitor2DSpatial_heat
from PINN.parameters import *


In [2]:
parser = argparse.ArgumentParser(description='PyTorch Training Heat Forward')

# 添加命令行参数
parser.add_argument('--lr', type=float, default=0.00001, help='Learning rate')
parser.add_argument('--batch_size', type=int, default=1024, help='Batch size')
parser.add_argument('--epochs', type=int, default=1000000, help='Total training epochs')
parser.add_argument('--gpu', type=bool , default=False ,help='If use GPU training')
parser.add_argument('--train_rec_size', type=int , default=128 ,help='Points generated in rectangle 128*128')
parser.add_argument('--train_bound_size', type=int , default=64 ,help='Points generated in each boundary')
parser.add_argument('--train_gen_random', type=bool , default=True ,help='If random generate points')
# BRR-PINN
parser.add_argument('--weight_equ1', type=int , default=1 ,help='equation weight')
parser.add_argument('--weight_equ2', type=int , default=5 ,help='equation weight') 
parser.add_argument('--weight_equ3', type=int , default=2 ,help='equation weight')  
parser.add_argument('--weight_equ4', type=int , default=5 ,help='equation weight') 
#PINN
parser.add_argument('--weight_up', type=int , default=1 ,help='Up boundary weight')
parser.add_argument('--weight_left', type=int , default=5 ,help='Left boundary weight')
parser.add_argument('--weight_right', type=int , default=1 ,help='Right boundary weight')
parser.add_argument('--weight_bottom', type=int , default=1 ,help='Bottom boundary weight')

parser.add_argument('--brr', type=int , default=30 ,help=' BRR coefficient')
parser.add_argument('--center_value', type=int , default=1 ,help='Center brr coefficient')
parser.add_argument('--network_MLP', type=str , default="(32,32,32,32,32)" ,help='Network shape')
parser.add_argument('--check_every', type=int , default=1000 ,help='Check period')
parser.add_argument('--save_dict', type=str , default='train_result/brrpinn_heat' ,help='Save dictionary')
parser.add_argument('--impose', type=int , default=1 ,help='If exactly impose boundary')



args = parser.parse_args(args=[])
#args = parser.parse_args() #If use .py file to train
print(args)

save_image_folder = args.save_dict + "-image/"
save_model_folder=args.save_dict + "-model/"

if not os.path.exists(save_image_folder):
    os.makedirs(save_image_folder)
if not os.path.exists(save_model_folder):
    os.makedirs(save_model_folder)

use_gpu = args.gpu
device = torch.device("cuda" if use_gpu else "cpu")
if use_gpu:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}

Namespace(lr=1e-05, batch_size=1024, epochs=1000000, gpu=False, train_rec_size=128, train_bound_size=64, train_gen_random=True, weight_equ1=1, weight_equ2=5, weight_equ3=2, weight_equ4=5, weight_up=1, weight_left=5, weight_right=1, weight_bottom=1, brr=30, center_value=1, network_MLP='(32,32,32,32,32)', check_every=1000, save_dict='train_result/brrpinn_heat', impose=1)


In [3]:
def heat_transfer(u , xx, yy):
    return diff(u, xx, order=2) + diff(u, yy, order=2)

# Normalize varibales xx,yy,u to range(0,1)
def heat_transfer_norm(u,xx,yy):
    return diff(u,xx)+((r2-r1)*xx+r1)/(r2-r1)*diff(u,xx,order=2)+((r2-r1)*xx+r1)*(r2-r1)/h1/h1*diff(u,yy,order=2)

def left(u,xx,yy):
    return diff(u,xx)

def bottom(u,xx,yy):
    return diff(u,yy)

def right(u,xx,yy):
    return (r2-r1)*h/k*u+diff(u,xx)

#left
adiabatic_left=BoundaryCondition(
    form=lambda u, x, y: diff(u,x),
    points_generator=generator_2dspatial_segment(size=args.train_bound_size, start=(0.0, 0.0), end=(0.0, 1.0),device=device),
    weight=args.weight_left,
    impose=0
)
#bottom
adiabatic_bottom=BoundaryCondition(
    form=lambda u, x, y: diff(u,y),
    #form=lambda u, x, y: u,
    points_generator=generator_2dspatial_segment(size=args.train_bound_size, start=(0.0, 0.0), end=(1.0, 0.0),device=device),
    weight=args.weight_bottom,
    impose=0
)
#right
convection_externel=BoundaryCondition(

    form=lambda u, x, y: (r2-r1)*h/k*u+diff(u,x),
    points_generator=generator_2dspatial_segment(size=args.train_bound_size, start=(1.0, 0.0), end=(1.0, 1.0),device=device),
    weight=args.weight_right,
    impose=0
)
#up
constant_interface=BoundaryCondition(
    form=lambda u, x, y: u-maxf*(2*x*x*x-3*x*x+1),
    points_generator=generator_2dspatial_segment(size=args.train_bound_size, start=(0.0, 1.0), end=(1.0, 1.0),device=device,random=True),
    weight=args.weight_up,
    impose=1
)

# Monitor metrics using to evaluate results during training
metrics={}

def pdemse(uu,xx,yy):
    error=heat_transfer_norm(uu[:,0],xx,yy)
    return torch.mean(abs(error)**2)
metrics['pdemse']=pdemse

# left boundary
def leftbound_mse(uu,xx,yy):
    x,y=next(adiabatic_left.points_generator)
    u=fcnn_approximator.__call__(x.requires_grad_(),y.requires_grad_())
    error=adiabatic_left.form(u[:,0],x,y)
    return torch.mean(abs(error)**2)
metrics['leftbound_mse']=leftbound_mse

# bottom boundary
def bottombound_mse(uu,xx,yy):
    x,y=next(adiabatic_bottom.points_generator)
    u=fcnn_approximator.__call__(x.requires_grad_(),y.requires_grad_())
    error=adiabatic_bottom.form(u[:,0],x,y)
    return torch.mean(abs(error)**2)
metrics['bottombound_mse']=bottombound_mse

# right boundary
def rightbound_mse(uu,xx,yy):
    x,y=next(convection_externel.points_generator)
    u=fcnn_approximator.__call__(x.requires_grad_(),y.requires_grad_())
    error=convection_externel.form(u[:,0],x,y)
    return torch.mean(abs(error)**2)
metrics['rightbound_mse']=rightbound_mse
    
# up boundary
def upbound_mse(uu,xx,yy):
    x,y=next(constant_interface.points_generator)
    u=fcnn_approximator.__call__(x.requires_grad_(),y.requires_grad_())
    error=constant_interface.form(u[:,0],x,y)
    return torch.mean(abs(error)**2)
metrics['upbound_mse']=upbound_mse

# compare with FEM result
def comsol_compare(uu,xx,yy):
    x=torch.linspace(0.0, 1.0, 400).requires_grad_()
    y=torch.linspace(0.0, 1.0, 400).requires_grad_()
    xy_tensor = torch.cartesian_prod(x, y).to(device)
    xx = torch.squeeze(xy_tensor[:, 0])
    yy = torch.squeeze(xy_tensor[:, 1])

    uu=fcnn_approximator.__call__(xx,yy)
    uu=uu.detach().cpu().numpy()
    fem=pd.read_csv('./fem_data/heat.txt',delimiter=r'\s+')
    uu_di=abs(uu[:,0]*maxf+303.15-fem['T'].values)
    return np.mean(uu_di **2 )
metrics['comsol_compare']=comsol_compare

fcnn = FCNN(
    n_input_units=2,
    n_output_units=4,
    hidden_units=ast.literal_eval(args.network_MLP),
    actv=nn.Tanh
)
fcnn=fcnn.to(device)


fcnn_approximator = SingleNetworkApproximator2DSpatial_heat(
    single_network=fcnn,
    #single_network=renn,
    pde=(heat_transfer_norm,left,bottom,right),
    boundary_conditions=[
        adiabatic_left,
        adiabatic_bottom,
        convection_externel,
        constant_interface
    ],
    args=args
)
adam = optim.Adam(fcnn_approximator.parameters(), lr=args.lr)
train_gen_spatial = generator_2dspatial_rectangle(size=(args.train_rec_size, args.train_rec_size), x_min=0.0, x_max=1.0, y_min=0.0, y_max=1.0,device=device,random=args.train_gen_random,bound=True)

/Users/yst/anaconda3/envs/DeepLearning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#%matplotlib inline
heat_transfer_2d_solution, _ = _solve_2dspatial(
    train_generator_spatial=train_gen_spatial,
    approximator=fcnn_approximator,
    optimizer=adam,
    batch_size=args.batch_size,
    max_epochs=args.epochs,
    shuffle=True,
    metrics=metrics,
    monitor=Monitor2DSpatial_heat(        
        check_on_x=torch.linspace(0.0, 1.0, 50),
        check_on_y=torch.linspace(0.0, 1.0, 50),
        check_every=args.check_every,
        device=device,
        args=args
    ),
    device=device,
    args=args
)